In [ ]:
!pip -q install sastrawi
!pip -q install nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import random
from joblib import dump
#from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset_terbaru.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   teks    184 non-null    object
 1   label   184 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


In [ ]:
def casefolding(text):
  text = text.lower()
  text = re.sub(r'https?://\S+|www\.\S+', '', text)
  text = re.sub(r'[-+]?[0-9]+', '', text)
  text = re.sub(r'[^\w\s]','', text)
  text = text.strip()
  return text

In [ ]:
raw_sample   = data['teks'].iloc[50]
case_folding = casefolding(raw_sample)

print('Raw data\t: ', raw_sample)
print('Case folding\t: ', case_folding)

Raw data	:  akses ke halaman kelas
Case folding	:  akses ke halaman kelas


In [ ]:
special_stemming_rules = {
    'di mana': 'dimana',
  'dimana': 'mana',
  'home': 'beranda',
  'awal': 'beranda',
  'utama': 'beranda',
  'depan': 'beranda',
  'course': 'kelas',
  'raport': 'rapor',
  'rapot': 'rapor',
  'leaderboard': 'peringkat',
  'load ulang': 'refresh',
  'perbarui': 'refresh',
  'muat ulang': 'refresh',
  'reload': 'refresh',
  'segarkan': 'refresh',
    # tambahkan aturan khusus lainnya di sini jika diperlukan
}

In [ ]:
#key_norm = pd.read_csv('key_norm.csv')

factory = StemmerFactory()
stemmer = factory.create_stemmer()

key_norm = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/key_norm.csv')  # Pastikan Anda sudah mengganti nama file sesuai dengan yang Anda gunakan

def text_normalize(text):
    # Mengganti singkatan
    text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])

    # Mengganti kata-kata sesuai dengan aturan khusus
    text = ' '.join([special_stemming_rules.get(word, word) for word in text.split()])

    # Mengonversi ke huruf kecil
    text = str.lower(text)

    return text

In [ ]:
stopwords_ind = stopwords.words('indonesian')
stopwords_ind = [word for word in stopwords_ind if word != 'mana']
more_stopword = ['ya']
stopwords_ind = stopwords_ind + more_stopword

In [ ]:
import json

# Simpan daftar stopwords dalam file JSON
with open('/content/drive/MyDrive/Colab Notebooks/tfjs_model/stopwords_indonesia.json', 'w') as json_file:
    json.dump(list(stop_words_indonesia), json_file)


In [ ]:
len(stopwords_ind)

758

In [ ]:
# Lihat daftar stopword yang disediakan NLTK
print(stopwords_ind)

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [ ]:
def remove_stop_words(text):
  clean_words = []
  text = text.split()
  for word in text:
      if word not in stopwords_ind:
          clean_words.append(word)
  return " ".join(clean_words)

In [ ]:
raw_sample       = data['teks'].iloc[5]
case_folding     = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)

print('Raw data\t\t: ', raw_sample)
print('Case folding\t\t: ', case_folding)
print('Stopword removal\t: ', stopword_removal)

Raw data		:  lihat beranda
Case folding		:  lihat beranda
Stopword removal	:  lihat beranda


In [ ]:
def stemming(text):
  text = stemmer.stem(text)
  return text

In [ ]:
raw_sample       = data['teks'].iloc[5]
case_folding     = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)
text_stemming    = stemming(stopword_removal)

print('Raw data\t\t: ', raw_sample)
print('Case folding\t\t: ', case_folding)
print('Stopword removal\t: ', stopword_removal)
print('Stemming\t\t: ', text_stemming)

Raw data		:  lihat beranda
Case folding		:  lihat beranda
Stopword removal	:  lihat beranda
Stemming		:  lihat beranda


In [ ]:
def text_preprocessing_process(text):
  text = casefolding(text)
  text = text_normalize(text)
  text = remove_stop_words(text)
  text = stemming(text)
  return text

In [ ]:
%%time
data['clean_teks'] = data['teks'].apply(text_preprocessing_process)

CPU times: user 1.65 s, sys: 14.3 ms, total: 1.66 s
Wall time: 1.71 s


In [ ]:
data

,teks,label,clean_teks
0,beranda,pergi beranda,beranda
1,utama,pergi beranda,beranda
2,pilih beranda,pergi beranda,pilih beranda
3,kembali ke beranda,pergi beranda,beranda
4,pergi ke halaman utama,pergi beranda,pergi halaman beranda
...,...,...,...
179,Saya perlu me-refresh.,muat ulang halaman,merefresh
180,Segarkan halaman web.,muat ulang halaman,refresh halaman situs
181,Akses fitur load ulang.,muat ulang halaman,akses fitur load ulang
182,Buka menu refresh.,muat ulang halaman,buka menu refresh


In [ ]:
# Simpan data yang telah melalui text preprocessing agar kita tidak perlu menjalankan proses tersebut mulai awal (Opsional)
data.to_csv('clean_data.csv')

In [ ]:
X = data['clean_teks']
y = data['label']

In [ ]:
X

0                     beranda
1                     beranda
2               pilih beranda
3                     beranda
4       pergi halaman beranda
                ...          
179                 merefresh
180     refresh halaman situs
181    akses fitur load ulang
182         buka menu refresh
183      refresh tampil situs
Name: clean_teks, Length: 184, dtype: object

In [ ]:
y

0           pergi beranda
1           pergi beranda
2           pergi beranda
3           pergi beranda
4           pergi beranda
              ...        
179    muat ulang halaman
180    muat ulang halaman
181    muat ulang halaman
182    muat ulang halaman
183    muat ulang halaman
Name: label, Length: 184, dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Inisialisasi objek LabelEncoder
label_encoder = LabelEncoder()

# Fit dan transform label unik
encoded_labels = label_encoder.fit_transform(data['label'])

# Tampilkan hasil
result_df = pd.DataFrame({
    'original_label': data['label'],
    'encoded_label': encoded_labels
})


# Tampilkan DataFrame hasil encoding
result_df

,original_label,encoded_label
0,pergi beranda,2
1,pergi beranda,2
2,pergi beranda,2
3,pergi beranda,2
4,pergi beranda,2
...,...,...
179,muat ulang halaman,1
180,muat ulang halaman,1
181,muat ulang halaman,1
182,muat ulang halaman,1


In [ ]:
z = pd.DataFrame(encoded_labels)
z

,0
0,2
1,2
2,2
3,2
4,2
...,...
179,1
180,1
181,1
182,1


In [ ]:
# Buat dictionary baru
label_dict = dict(zip(result_df['encoded_label'], result_df['original_label']))

# Urutkan dictionary berdasarkan nilai (encoded_label)
sorted_label_dict = dict(sorted(label_dict.items(), key=lambda item: item[1]))

# Tampilkan dictionary yang telah diurutkan
print(sorted_label_dict)


{0: 'jelaskan instruksi', 1: 'muat ulang halaman', 2: 'pergi beranda', 3: 'pergi kelas', 4: 'pergi peringkat', 5: 'pergi rapor', 6: 'saya dimana'}


In [ ]:
# Konversi kamus ke JSON
json_data = json.dumps(sorted_label_dict, indent=2)

# Menyimpan JSON ke file
with open('/content/drive/MyDrive/Colab Notebooks/tfjs_model/label_encoder.json', 'w') as file:
    file.write(json_data)

In [ ]:
X

0                     beranda
1                     beranda
2               pilih beranda
3                     beranda
4       pergi halaman beranda
                ...          
179                 merefresh
180     refresh halaman situs
181    akses fitur load ulang
182         buka menu refresh
183      refresh tampil situs
Name: clean_teks, Length: 184, dtype: object

In [ ]:
tf_idf = TfidfVectorizer(ngram_range=(1,1))
tf_idf.fit(X)

TfidfVectorizer()

In [ ]:
# Melihat Jumlah Fitur
fitur = len(tf_idf.get_feature_names_out())
print(fitur)

53


In [ ]:
# Melihat fitur-fitur apa saja yang ada di dalam corpus
print(tf_idf.get_feature_names_out())

['ada' 'akses' 'alih' 'arah' 'bantu' 'beranda' 'browser' 'buka' 'butuh'
 'cari' 'cek' 'fitur' 'go' 'guna' 'halaman' 'hasil' 'informasi'
 'instruksi' 'kambali' 'kelas' 'klik' 'kunjung' 'laman' 'layar' 'letak'
 'lihat' 'load' 'lokasi' 'mana' 'masuk' 'menu' 'merefresh' 'muat'
 'navigasi' 'open' 'paham' 'pandu' 'pergi' 'peringkat' 'pertatama' 'pilih'
 'pindah' 'posisi' 'rapor' 'refresh' 'situs' 'tab' 'tampil' 'taut'
 'tolong' 'tombol' 'tunjuk' 'ulang']


In [ ]:
X_tf_idf = tf_idf.transform(X).toarray()
X_tf_idf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.46053631, 0.        , ..., 0.        , 0.        ,
        0.47401918],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
# Melihat matriks jumlah token menggunakan TF IDF, lihat perbedaannya dengan metode BoW
# Data ini siap untuk dimasukkan dalam proses pemodelan (machine learning)

data_tf_idf = pd.DataFrame(X_tf_idf, columns=tf_idf.get_feature_names_out())
data_tf_idf

,ada,akses,alih,arah,bantu,beranda,browser,buka,butuh,cari,...,rapor,refresh,situs,tab,tampil,taut,tolong,tombol,tunjuk,ulang
0,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.0,0.0,0.0,0.432834,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.0,0.0,0.0,0.527007,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
180,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.656248,0.630557,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
181,0.0,0.460536,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.474019
182,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.438371,0.0,0.0,...,0.0,0.594813,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000


In [ ]:
with open('tf_idf_feature.pickle', 'wb') as output:
  pickle.dump(X_tf_idf, output)

In [ ]:
# Mengubah nilai data tabular tf-idf menjadi array agar dapat dijalankan pada proses seleksi fitur
X = np.array(data_tf_idf)
y = np.array(z)

In [ ]:
chi2_features = SelectKBest(chi2, k=fitur)
X_kbest_features = chi2_features.fit_transform(X_tf_idf, y)

In [ ]:
# Reduced features
print('Original feature number:', X.shape[1])
print('Reduced feature number:', X_kbest_features.shape[1])

Original feature number: 53
Reduced feature number: 53


In [ ]:
# y

In [ ]:
# chi2_features.scores_ adalah nilai chi-square, semakin tinggi nilainya maka semakin baik fiturnya
data_chi2 = pd.DataFrame(chi2_features.scores_, columns=['nilai'])
data_chi2

,nilai
0,8.792487
1,2.865692
2,7.151874
3,5.027390
4,49.268969
5,68.879576
6,11.815053
7,2.761714
8,6.296662
9,7.709694


In [ ]:
# Menampilkan fitur beserta nilainya
feature = tf_idf.get_feature_names_out()
data_chi2['fitur'] = feature
data_chi2

,nilai,fitur
0,8.792487,ada
1,2.865692,akses
2,7.151874,alih
3,5.027390,arah
4,49.268969,bantu
5,68.879576,beranda
6,11.815053,browser
7,2.761714,buka
8,6.296662,butuh
9,7.709694,cari


In [ ]:
# Mengurutkan fitur terbaik
sorted_data = data_chi2.sort_values(by='nilai', ascending=False)

In [ ]:
# Mengubah ke format kamus yang diinginkan
result_dict = {row['fitur']: int(row['nilai']) for index, row in sorted_data.iterrows()}

# Mengonversi kamus ke JSON
result_json = json.dumps(result_dict, indent=2)

# Menyimpan JSON ke file
with open('/content/drive/MyDrive/Colab Notebooks/tfjs_model/vocab.json', 'w') as file:
    file.write(result_json)
print(result_json)

{
  "peringkat": 99,
  "mana": 94,
  "rapor": 80,
  "beranda": 68,
  "kelas": 61,
  "refresh": 61,
  "bantu": 49,
  "posisi": 44,
  "hasil": 36,
  "ulang": 31,
  "instruksi": 27,
  "pandu": 22,
  "muat": 20,
  "tunjuk": 19,
  "cek": 16,
  "load": 15,
  "lihat": 14,
  "halaman": 13,
  "layar": 13,
  "browser": 11,
  "pergi": 11,
  "fitur": 10,
  "lokasi": 9,
  "letak": 9,
  "ada": 8,
  "open": 8,
  "merefresh": 8,
  "guna": 8,
  "cari": 7,
  "informasi": 7,
  "tolong": 7,
  "alih": 7,
  "masuk": 6,
  "paham": 6,
  "butuh": 6,
  "situs": 5,
  "kunjung": 5,
  "tombol": 5,
  "klik": 5,
  "navigasi": 5,
  "arah": 5,
  "go": 4,
  "kambali": 4,
  "tampil": 4,
  "pindah": 3,
  "akses": 2,
  "buka": 2,
  "pertatama": 2,
  "tab": 2,
  "laman": 2,
  "pilih": 2,
  "taut": 2,
  "menu": 1
}


In [ ]:
# Menampilkan mask pada feature yang diseleksi
# False berarti fitur tidak terpilih dan True berarti fitur terpilih
mask = chi2_features.get_support()
mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [ ]:
# Menampilkan fitur-fitur terpilih berdasarkan mask atau nilai tertinggi yang sudah dikalkulasi pada Chi-Square
new_feature = []

for bool, f in zip(mask, feature):
  if bool:
    new_feature.append(f)
  selected_feature = new_feature

selected_feature

['ada',
 'akses',
 'alih',
 'arah',
 'bantu',
 'beranda',
 'browser',
 'buka',
 'butuh',
 'cari',
 'cek',
 'fitur',
 'go',
 'guna',
 'halaman',
 'hasil',
 'informasi',
 'instruksi',
 'kambali',
 'kelas',
 'klik',
 'kunjung',
 'laman',
 'layar',
 'letak',
 'lihat',
 'load',
 'lokasi',
 'mana',
 'masuk',
 'menu',
 'merefresh',
 'muat',
 'navigasi',
 'open',
 'paham',
 'pandu',
 'pergi',
 'peringkat',
 'pertatama',
 'pilih',
 'pindah',
 'posisi',
 'rapor',
 'refresh',
 'situs',
 'tab',
 'tampil',
 'taut',
 'tolong',
 'tombol',
 'tunjuk',
 'ulang']

In [ ]:
# Memilih fitur yang telah terseleksi dari vocabulary TF IDF
kbest_feature = {}

for (k,v) in tf_idf.vocabulary_.items():
  if k in selected_feature:
    kbest_feature[k] = v

In [ ]:
tf_idf.vocabulary_.items()

dict_items([('beranda', 5), ('pilih', 40), ('pergi', 37), ('halaman', 14), ('lihat', 25), ('menu', 30), ('buka', 7), ('navigasi', 33), ('klik', 20), ('pindah', 41), ('tab', 46), ('masuk', 29), ('tampil', 47), ('situs', 45), ('kunjung', 21), ('taut', 48), ('arah', 3), ('pertatama', 39), ('akses', 1), ('laman', 22), ('kelas', 19), ('alih', 2), ('rapor', 43), ('go', 12), ('kambali', 18), ('open', 34), ('hasil', 15), ('peringkat', 38), ('cek', 10), ('guna', 13), ('mana', 28), ('posisi', 42), ('lokasi', 27), ('ada', 0), ('letak', 24), ('tolong', 49), ('informasi', 16), ('cari', 9), ('instruksi', 17), ('pandu', 36), ('fitur', 11), ('bantu', 4), ('tunjuk', 51), ('butuh', 8), ('paham', 35), ('muat', 32), ('ulang', 52), ('refresh', 44), ('layar', 23), ('load', 26), ('browser', 6), ('tombol', 50), ('merefresh', 31)])

In [ ]:
kbest_feature

{'beranda': 5,
 'pilih': 40,
 'pergi': 37,
 'halaman': 14,
 'lihat': 25,
 'menu': 30,
 'buka': 7,
 'navigasi': 33,
 'klik': 20,
 'pindah': 41,
 'tab': 46,
 'masuk': 29,
 'tampil': 47,
 'situs': 45,
 'kunjung': 21,
 'taut': 48,
 'arah': 3,
 'pertatama': 39,
 'akses': 1,
 'laman': 22,
 'kelas': 19,
 'alih': 2,
 'rapor': 43,
 'go': 12,
 'kambali': 18,
 'open': 34,
 'hasil': 15,
 'peringkat': 38,
 'cek': 10,
 'guna': 13,
 'mana': 28,
 'posisi': 42,
 'lokasi': 27,
 'ada': 0,
 'letak': 24,
 'tolong': 49,
 'informasi': 16,
 'cari': 9,
 'instruksi': 17,
 'pandu': 36,
 'fitur': 11,
 'bantu': 4,
 'tunjuk': 51,
 'butuh': 8,
 'paham': 35,
 'muat': 32,
 'ulang': 52,
 'refresh': 44,
 'layar': 23,
 'load': 26,
 'browser': 6,
 'tombol': 50,
 'merefresh': 31}

In [ ]:
# Mengurutkan dictionary berdasarkan kunci (abjad)
kbest_feature_sorted = dict(sorted(kbest_feature.items()))

# Menampilkan hasil
print(kbest_feature_sorted)

{'ada': 0, 'akses': 1, 'alih': 2, 'arah': 3, 'bantu': 4, 'beranda': 5, 'browser': 6, 'buka': 7, 'butuh': 8, 'cari': 9, 'cek': 10, 'fitur': 11, 'go': 12, 'guna': 13, 'halaman': 14, 'hasil': 15, 'informasi': 16, 'instruksi': 17, 'kambali': 18, 'kelas': 19, 'klik': 20, 'kunjung': 21, 'laman': 22, 'layar': 23, 'letak': 24, 'lihat': 25, 'load': 26, 'lokasi': 27, 'mana': 28, 'masuk': 29, 'menu': 30, 'merefresh': 31, 'muat': 32, 'navigasi': 33, 'open': 34, 'paham': 35, 'pandu': 36, 'pergi': 37, 'peringkat': 38, 'pertatama': 39, 'pilih': 40, 'pindah': 41, 'posisi': 42, 'rapor': 43, 'refresh': 44, 'situs': 45, 'tab': 46, 'tampil': 47, 'taut': 48, 'tolong': 49, 'tombol': 50, 'tunjuk': 51, 'ulang': 52}


In [ ]:
kbest_feature.items()

dict_items([('beranda', 5), ('pilih', 40), ('pergi', 37), ('halaman', 14), ('lihat', 25), ('menu', 30), ('buka', 7), ('navigasi', 33), ('klik', 20), ('pindah', 41), ('tab', 46), ('masuk', 29), ('tampil', 47), ('situs', 45), ('kunjung', 21), ('taut', 48), ('arah', 3), ('pertatama', 39), ('akses', 1), ('laman', 22), ('kelas', 19), ('alih', 2), ('rapor', 43), ('go', 12), ('kambali', 18), ('open', 34), ('hasil', 15), ('peringkat', 38), ('cek', 10), ('guna', 13), ('mana', 28), ('posisi', 42), ('lokasi', 27), ('ada', 0), ('letak', 24), ('tolong', 49), ('informasi', 16), ('cari', 9), ('instruksi', 17), ('pandu', 36), ('fitur', 11), ('bantu', 4), ('tunjuk', 51), ('butuh', 8), ('paham', 35), ('muat', 32), ('ulang', 52), ('refresh', 44), ('layar', 23), ('load', 26), ('browser', 6), ('tombol', 50), ('merefresh', 31)])

In [ ]:
import json

# Konversi kamus ke JSON
json_data = json.dumps(kbest_feature_sorted, indent=2)


In [ ]:
# Menampilkan fitur-fitur yang sudah diseleksi
# Beserta nilai vektornya pada keseluruhan data untuk dijalankan pada proses machine learning

# Hanya k fitur yang terpilih sesuai parameter k yang ditentukan sebelumnya

data_selected_feature = pd.DataFrame(X_kbest_features, columns=selected_feature)
data_selected_feature

,ada,akses,alih,arah,bantu,beranda,browser,buka,butuh,cari,...,rapor,refresh,situs,tab,tampil,taut,tolong,tombol,tunjuk,ulang
0,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.0,0.0,0.0,0.432834,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.0,0.0,0.0,0.527007,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
180,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.656248,0.630557,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
181,0.0,0.460536,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.474019
182,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.438371,0.0,0.0,...,0.0,0.594813,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000


In [ ]:
with open('kbest_feature.pickle', 'wb') as output:
  pickle.dump(kbest_feature, output)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_kbest_features, y, test_size=0.1, random_state=0)

In [ ]:
y_train

array([[3],
       [1],
       [0],
       [3],
       [1],
       [0],
       [2],
       [3],
       [6],
       [4],
       [4],
       [4],
       [3],
       [4],
       [2],
       [2],
       [6],
       [5],
       [2],
       [3],
       [5],
       [3],
       [4],
       [3],
       [2],
       [2],
       [5],
       [6],
       [5],
       [5],
       [2],
       [1],
       [6],
       [0],
       [2],
       [3],
       [5],
       [4],
       [2],
       [4],
       [2],
       [6],
       [1],
       [6],
       [4],
       [5],
       [4],
       [2],
       [1],
       [1],
       [4],
       [1],
       [5],
       [3],
       [2],
       [3],
       [5],
       [5],
       [2],
       [2],
       [5],
       [3],
       [1],
       [1],
       [1],
       [4],
       [5],
       [0],
       [1],
       [3],
       [4],
       [5],
       [3],
       [6],
       [2],
       [3],
       [3],
       [3],
       [5],
       [2],
       [6],
       [2],
       [1],
    

In [ ]:
X_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.61158427],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(fitur,)),
    #keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(7, activation='softmax')
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Assuming y_train and y_test are your label data
model.compile(optimizer=RMSprop(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/200
3/3 [==============================] - 1s 109ms/step - loss: 1.8931 - accuracy: 0.2424 - val_loss: 1.8512 - val_accuracy: 0.2632
Epoch 2/200
3/3 [==============================] - 0s 16ms/step - loss: 1.8101 - accuracy: 0.2667 - val_loss: 1.7984 - val_accuracy: 0.3684
Epoch 3/200
3/3 [==============================] - 0s 17ms/step - loss: 1.7457 - accuracy: 0.4242 - val_loss: 1.7441 - val_accuracy: 0.4211
Epoch 4/200
3/3 [==============================] - 0s 17ms/step - loss: 1.6808 - accuracy: 0.4909 - val_loss: 1.6925 - val_accuracy: 0.5263
Epoch 5/200
3/3 [==============================] - 0s 18ms/step - loss: 1.6154 - accuracy: 0.5879 - val_loss: 1.6335 - val_accuracy: 0.6316
Epoch 6/200
3/3 [==============================] - 0s 17ms/step - loss: 1.5503 - accuracy: 0.6424 - val_loss: 1.5771 - val_accuracy: 0.5263
Epoch 7/200
3/3 [==============================] - 0s 19ms/step - loss: 1.4793 - accuracy: 0.6545 - val_loss: 1.5163 - val_accuracy: 0.7368
Epoch 8/200
3/3 [==

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/teks_klasifikasi.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  for trackable_obj in self._self_tracked_trackables:


In [ ]:
# Prediksi untuk Data Latih
predicted_train_probs = model.predict(X_train)
predicted_train = np.argmax(predicted_train_probs, axis=1)

6/6 [==============================] - 0s 5ms/step


In [ ]:
y_train

array([[3],
       [1],
       [0],
       [3],
       [1],
       [0],
       [2],
       [3],
       [6],
       [4],
       [4],
       [4],
       [3],
       [4],
       [2],
       [2],
       [6],
       [5],
       [2],
       [3],
       [5],
       [3],
       [4],
       [3],
       [2],
       [2],
       [5],
       [6],
       [5],
       [5],
       [2],
       [1],
       [6],
       [0],
       [2],
       [3],
       [5],
       [4],
       [2],
       [4],
       [2],
       [6],
       [1],
       [6],
       [4],
       [5],
       [4],
       [2],
       [1],
       [1],
       [4],
       [1],
       [5],
       [3],
       [2],
       [3],
       [5],
       [5],
       [2],
       [2],
       [5],
       [3],
       [1],
       [1],
       [1],
       [4],
       [5],
       [0],
       [1],
       [3],
       [4],
       [5],
       [3],
       [6],
       [2],
       [3],
       [3],
       [3],
       [5],
       [2],
       [6],
       [2],
       [1],
    

In [ ]:
print("Train Accuracy:", accuracy_score(y_train, predicted_train))

Train Accuracy: 1.0


In [ ]:
# Prediksi untuk Data Uji
predicted_test_probs = model.predict(X_test)
predicted_test = np.argmax(predicted_test_probs, axis=1)

1/1 [==============================] - 0s 31ms/step


In [ ]:
y_test

array([[6],
       [5],
       [2],
       [5],
       [3],
       [5],
       [3],
       [1],
       [2],
       [1],
       [0],
       [3],
       [2],
       [4],
       [0],
       [0],
       [2],
       [3],
       [0]])

In [ ]:
print("Test Accuracy:", accuracy_score(y_test, predicted_test))

Test Accuracy: 1.0


In [ ]:
print("\nClassification Report for Test Data:\n", classification_report(y_test, predicted_test))


Classification Report for Test Data:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         1

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19



In [ ]:
# save the model to disk
#dump(model, filename="model.json")

In [ ]:
# Simpan bobot model ke file
model.save_weights("/content/drive/MyDrive/Colab Notebooks/model_weights_baru.h5")


In [ ]:
from tensorflow.keras.models import load_model

# Memuat kembali model dari file
model = load_model("/content/drive/MyDrive/Colab Notebooks/teks_klasifikasi.h5")

In [ ]:
print(model)

In [ ]:
#@title Masukkan Teks Anda:
input_text = "pergi beranda" #@param {type:"string"}

pre_input_text = text_preprocessing_process(input_text)   # lakukan text pre processing pada text input

tf_idf_vec = TfidfVectorizer(vocabulary=set(vocab))       # definisikan TF_IDF
print( tf_idf_vec)
# Ubah data TF-IDF menjadi bentuk dense array
tf_idf_dense = tf_idf_vec.fit_transform([pre_input_text]).toarray()
print(tf_idf_dense)
print(type(tf_idf_dense))

result = model.predict(tf_idf_dense)  # Lakukan prediksi

# Ambil nilai argmax dari hasil prediksi
predicted_label_index = result.argmax()

# Cek apakah nilai argmax ada dalam label_dict
if np.sum(tf_idf_dense) == 0:
  print("Label tidak ditemukan dalam label_dict.")
else:
  print(label_dict[predicted_label_index])


TfidfVectorizer(vocabulary={'ada', 'akses', 'alih', 'arah', 'bantu', 'beranda',
                            'browser', 'buka', 'butuh', 'cari', 'cek', 'fitur',
                            'go', 'guna', 'halaman', 'hasil', 'informasi',
                            'instruksi', 'kambali', 'kelas', 'klik', 'kunjung',
                            'laman', 'layar', 'letak', 'lihat', 'load',
                            'lokasi', 'mana', 'masuk', ...})
[[0.         0.         0.         0.         0.         0.70710678
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.70710678 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]]
<clas

1/1 [==============================] - 0s 172ms/step
pergi beranda


In [ ]:
# Install TensorFlow.js Converter
!pip install tensorflowjs

In [ ]:


# Import Libraries
import tensorflow as tf
from tensorflowjs import converters
from google.colab import files

# Muat Model Keras
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/teks_klasifikasi.h5')

# Tentukan path untuk menyimpan model TensorFlow.js
model_path = '/content/drive/MyDrive/Colab Notebooks/tfjs_model'

# Konversi ke TensorFlow.js
converters.save_keras_model(model, model_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  for trackable_obj in self._self_tracked_trackables:
